# Visualizing Metop and MSG data on the same grid
## Summary
In this tutorial, we well look at how we can put both Metop and MSG data on the same grid

## First step, loading the data
Loading the data is done in the same way as presented in earlier tutorial

## Exercise 6.1: Getting the data on the same grid
Create a `Scene` object for the Metop data and another one for the MSG data. Load the `day_microphysics` RGB composite from MSG and the 10.8µm channel from Metop. Resample both to the EastEurope area.

In [ ]:
from satpy import Scene, find_files_and_readers
from datetime import datetime

metop_files = find_files_and_readers(base_dir='/tcenas/scratch/pytroll/ex6',
                                     reader='avhrr_eps_l1b',
                                     start_time=datetime(2018, 10, 7 ,9, 25),
                                     end_time=datetime(2018, 10, 7 ,9, 30))
scn_metop = Scene(filenames=metop_files)
scn_metop.load([10.8])

In [ ]:
msg_files = find_files_and_readers(base_dir='/tcenas/scratch/pytroll/ex6',
                                   reader='native_msg')

scn = Scene(filenames=msg_files)
scn_msg = Scene(filenames=msg_files)
scn_msg.load(['day_microphysics','IR_108'])


In [ ]:
newscn_metop = scn_metop.resample('eurol')
newscn_metop.show(10.8)

In [ ]:

newscn_msg = scn_msg.resample('eurol')
newscn_msg.show('IR_108')


## Exercise 6.2:  Manipulating images
Now we want to put both datasets onto one image.

In [ ]:
from satpy.writers import get_enhanced_image
import xarray as xr
from trollimage.xrimage import XRImage

# Lets compare MSG and METOP channel 10.8 data
msg_image = get_enhanced_image(newscn_msg['IR_108'])

metop_image = get_enhanced_image(newscn_metop[10.8])

array1 = msg_image.data.where(metop_image.data.isnull(), metop_image.data)

XRImage(array1)




In [ ]:
# No Using MSG composite
msg_image = get_enhanced_image(newscn_msg['day_microphysics'])
color_array = xr.concat((metop_image.data, metop_image.data, metop_image.data), 'bands')
color_array['bands'] = ['R', 'G', 'B']
final_array = msg_image.data.where(color_array.isnull(), color_array.data)
XRImage(final_array)

## The `Multiscene` class
In order to manipulate multiple scenes at once, in some circumstances, it maybe advantageous to make use of satpy's `Multiscene` class. Let's create a multiscene instance, load some data and resample the data to the `EastEurope` area

In [ ]:
from satpy import MultiScene
mscn = MultiScene([scn_msg, scn_metop])
mscn.load(['overview'])
new_mscn = mscn.resample('eurol')
blended_scene = new_mscn.blend()
blended_scene.show('overview', overlay={'coast_dir': '/tcenas/scratch/pytroll/shapes/', 'color': (255, 0, 0), 'resolution': 'i'})